OPR700 - Create Grafana Dashboard for App-Deploy Applications
=============================================================

Description
-----------

This notebook will create a dashboard in Grafana to monitor App-Deploy
results and generate alerts if a Canary or Application starts failing.

To receive these Alerts, set up a `Notification Channel` in Grafana

-   https://{metricsui\_host:30777}/grafana/alerting/notifications

If your team uses `Microsoft Teams`, creating a notificaiton channel to
use it is easy to configure (with an `incoming webhook` url).

-   https://docs.microsoft.com/en-us/microsoftteams/platform/webhooks-and-connectors/how-to/add-incoming-webhook

To provide the data for the dashabord, this notebook will also create a
`datasource` in Grafana for the SQL Server master pool in the Big Data
Cluster that contains the `runner` database hosting the metrics from the
canary results.

### Parameters

Set the `alert` threshold.

In [ ]:
alert_threshold = 0.9

database_name = "runner"

test_cert_store_root = "/var/opt/secrets/test-certificates"

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import json
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

first_run = True
rules = None
debug_logging = False

def run(cmd, return_output=False, no_output=False, retry_count=0):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    global first_run
    global rules

    if first_run:
        first_run = False
        rules = load_rules()

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportabilty, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    if which_binary == None:
        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    print(f"START: {cmd} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')
                            if rules is not None:
                                apply_expert_rules(line)

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # apply expert rules (to run follow-on notebooks), based on output
                #
                if rules is not None:
                    apply_expert_rules(line_decoded)

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    return output
                                else:
                                    return

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        return output

def load_json(filename):
    """Load a json file from disk and return the contents"""

    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

def load_rules():
    """Load any 'expert rules' from the metadata of this notebook (.ipynb) that should be applied to the stderr of the running executable"""

    # Load this notebook as json to get access to the expert rules in the notebook metadata.
    #
    try:
        j = load_json("opr700-create-app-dashboard.ipynb")
    except:
        pass # If the user has renamed the book, we can't load ourself.  NOTE: Is there a way in Jupyter, to know your own filename?
    else:
        if "metadata" in j and \
            "azdata" in j["metadata"] and \
            "expert" in j["metadata"]["azdata"] and \
            "expanded_rules" in j["metadata"]["azdata"]["expert"]:

            rules = j["metadata"]["azdata"]["expert"]["expanded_rules"]

            rules.sort() # Sort rules, so they run in priority order (the [0] element).  Lowest value first.

            # print (f"EXPERT: There are {len(rules)} rules to evaluate.")

            return rules

def apply_expert_rules(line):
    """Determine if the stderr line passed in, matches the regular expressions for any of the 'expert rules', if so
    inject a 'HINT' to the follow-on SOP/TSG to run"""

    global rules

    for rule in rules:
        notebook = rule[1]
        cell_type = rule[2]
        output_type = rule[3] # i.e. stream or error
        output_type_name = rule[4] # i.e. ename or name 
        output_type_value = rule[5] # i.e. SystemExit or stdout
        details_name = rule[6]  # i.e. evalue or text 
        expression = rule[7].replace("\\*", "*") # Something escaped *, and put a \ in front of it!

        if debug_logging:
            print(f"EXPERT: If rule '{expression}' satisfied', run '{notebook}'.")

        if re.match(expression, line, re.DOTALL):

            if debug_logging:
                print("EXPERT: MATCH: name = value: '{0}' = '{1}' matched expression '{2}', therefore HINT '{4}'".format(output_type_name, output_type_value, expression, notebook))

            match_found = True

            display(Markdown(f'HINT: Use [{notebook}]({notebook}) to resolve this issue.'))




print('Common functions defined successfully.')

# Hints for binary (transient fault) retry, (known) error and install guide
#
retry_hints = {'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'], 'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use']}
error_hints = {'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb']], 'azdata': [['azdata login', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Error processing command: "ApiError', 'TSG110 - Azdata returns ApiError', '../repair/tsg110-azdata-returns-apierror.ipynb'], ['Error processing command: "ControllerError', 'TSG036 - Controller logs', '../log-analyzers/tsg036-get-controller-logs.ipynb'], ['ERROR: 500', 'TSG046 - Knox gateway logs', '../log-analyzers/tsg046-get-knox-logs.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ["Can't open lib 'ODBC Driver 17 for SQL Server", 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb']]}
install_hint = {'kubectl': ['SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb'], 'azdata': ['SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb']}

### Get the Kubernetes namespace for the big data cluster

Get the namespace of the Big Data Cluster use the kubectl command line
interface .

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = run(f'kubectl get namespace --selector=MSSQL_CLUSTER -o jsonpath={{.items[0].metadata.name}}', return_output=True)
    except:
        from IPython.display import Markdown
        print(f"ERROR: Unable to find a Kubernetes namespace with label 'MSSQL_CLUSTER'.  SQL Server Big Data Cluster Kubernetes namespaces contain the label 'MSSQL_CLUSTER'.")
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print(f'The SQL Server Big Data Cluster Kubernetes namespace is: {namespace}')

### Place `runner-db-reader-login` secret in AZDATA\_USERNAME/AZDATA\_PASSWORD environment variables

In [ ]:
import os, base64

os.environ["AZDATA_USERNAME"] = run(f'kubectl get secret/runner-db-reader-login-secret -n {namespace} -o jsonpath={{.data.username}}', return_output=True)
os.environ["AZDATA_USERNAME"] = base64.b64decode(os.environ["AZDATA_USERNAME"]).decode('utf-8')

os.environ["AZDATA_PASSWORD"] = run(f'kubectl get secret/runner-db-reader-login-secret -n {namespace} -o jsonpath={{.data.password}}', return_output=True)
os.environ["AZDATA_PASSWORD"] = base64.b64decode(os.environ["AZDATA_PASSWORD"]).decode('utf-8')

print(f"runner-db-reader-login username '{os.environ['AZDATA_USERNAME']}' and password stored in environment variables")

### Define Master Pool as Datasource

NOTE: The “Json Model” in Grafana uses lowercase `false` and `true` and
the `null` keyword, which is not valid as a Python data type value

In [ ]:
false = False
true = True
null = None

datasource = {
    "id": null,
    "orgId":1,
    "name":"Microsoft SQL Server",
    "type":"mssql",
    "typeLogoUrl":"",
    "access":"proxy",
    "url":"master-p-svc",
    "password":"",
    "user":os.environ["AZDATA_USERNAME"],
    "database":database_name,
    "basicAuth":false,
    "basicAuthUser":"",
    "basicAuthPassword":"",
    "withCredentials":false,
    "isDefault":false,
    "jsonData": { 
      "encrypt":"false" },
    "secureJsonFields": { 
      "password":true },
    "secureJsonData": {
      "password":os.environ["AZDATA_PASSWORD"] },
    "readOnly":false
}

### Get the controller username and password

Get the controller username and password from the Kubernetes Secret
Store and place in the required AZDATA\_USERNAME and AZDATA\_PASSWORD
environment variables.

In [ ]:
# Place controller secret in AZDATA_USERNAME/AZDATA_PASSWORD environment variables

import os, base64

os.environ["AZDATA_USERNAME"] = run(f'kubectl get secret/controller-login-secret -n {namespace} -o jsonpath={{.data.username}}', return_output=True)
os.environ["AZDATA_USERNAME"] = base64.b64decode(os.environ["AZDATA_USERNAME"]).decode('utf-8')

os.environ["AZDATA_PASSWORD"] = run(f'kubectl get secret/controller-login-secret -n {namespace} -o jsonpath={{.data.password}}', return_output=True)
os.environ["AZDATA_PASSWORD"] = base64.b64decode(os.environ["AZDATA_PASSWORD"]).decode('utf-8')


print(f"Controller username '{os.environ['AZDATA_USERNAME']}' and password stored in environment variables")

### Get the webhdfs endpoint

The webdhfs endpoint is used to hyperlink to the notebooks in the
Storage Pool

In [ ]:
import json

endpoint = run('azdata bdc endpoint list --endpoint="webhdfs"', return_output=True)
endpoint = json.loads(endpoint)
webhdfs_endpoint = endpoint['endpoint']

### Define Dashboard

-   The dashboard definition can be retrieved from the “Dashboard
    Settings” “Json Model” menu option.

NOTE: The “Json Model” in Grafana uses lowercase `false` and `true` and
the `null` keyword, which is not valid as a Python data type value

In [ ]:
false = False
true = True
null = None

dashboard = {
  "annotations": {
    "list": [
      {
        "builtIn": 1,
        "datasource": "-- Grafana --",
        "enable": true,
        "hide": true,
        "iconColor": "rgba(0, 211, 255, 1)",
        "name": "Annotations & Alerts",
        "type": "dashboard"
      }
    ]
  },
  "editable": true,
  "gnetId": null,
  "graphTooltip": 0,
  "id": null,
  "links": [],
  "panels": [
    {
      "alert": {
        "conditions": [
          {
            "evaluator": {
              "params": [
                0.9
              ],
              "type": "lt"
            },
            "operator": {
              "type": "and"
            },
            "query": {
              "params": [
                "A",
                "5m",
                "now"
              ]
            },
            "reducer": {
              "params": [],
              "type": "avg"
            },
            "type": "query"
          }
        ],
        "executionErrorState": "alerting",
        "for": "5m",
        "frequency": "1m",
        "handler": 1,
        "message": f"Notebook App-Deploy failed in namespace: {namespace}",
        "name": f"Canary/Application ({namespace})",
        "noDataState": "alerting",
        "notifications": []
      },
      "aliasColors": {},
      "bars": false,
      "dashLength": 10,
      "dashes": false,
      "datasource": "Microsoft SQL Server",
      "description": "",
      "fill": 0,
      "gridPos": {
        "h": 8,
        "w": 24,
        "x": 0,
        "y": 0
      },
      "id": 28,
      "legend": {
        "alignAsTable": true,
        "avg": true,
        "current": true,
        "hideEmpty": false,
        "hideZero": false,
        "max": false,
        "min": false,
        "rightSide": true,
        "show": true,
        "total": false,
        "values": true
      },
      "lines": true,
      "linewidth": 2,
      "links": [
        {
          "includeVars": true,
          "keepTime": true,
          "targetBlank": true,
          "type": "dashboard"
        }
      ],
      "nullPointMode": "null",
      "percentage": false,
      "pointradius": 3,
      "points": true,
      "renderer": "flot",
      "seriesOverrides": [],
      "spaceLength": 10,
      "stack": false,
      "steppedLine": true,
      "targets": [
        {
          "alias": "",
          "format": "time_series",
          "rawSql": "SELECT\n  $__timeEpoch(session_start),\n  CASE error_level WHEN 0 THEN 1 ELSE 0 END as value,\n  app_name as metric\nFROM\n metrics\nWHERE\n  $__timeFilter(session_start)\nORDER BY\n  session_start ASC",
          "refId": "A"
        }
      ],
      "thresholds": [
        {
          "colorMode": "critical",
          "fill": true,
          "line": true,
          "op": "lt",
          "value": 0.9
        }
      ],
      "timeFrom": null,
      "timeRegions": [],
      "timeShift": null,
      "title": f"Notebook App-Deploy Status ({namespace})",
      "tooltip": {
        "shared": true,
        "sort": 0,
        "value_type": "individual"
      },
      "type": "graph",
      "xaxis": {
        "buckets": null,
        "mode": "time",
        "name": null,
        "show": true,
        "values": []
      },
      "yaxes": [
        {
          "decimals": null,
          "format": "short",
          "label": "Pass/Fail",
          "logBase": 1,
          "max": "1.1",
          "min": "0",
          "show": true
        },
        {
          "decimals": null,
          "format": "dtdurations",
          "label": "Duration",
          "logBase": 1,
          "max": "1",
          "min": "0",
          "show": false
        }
      ],
      "yaxis": {
        "align": false,
        "alignLevel": null
      }
    },
    {
      "columns": [],
      "datasource": "Microsoft SQL Server",
      "fontSize": "100%",
      "gridPos": {
        "h": 6,
        "w": 5,
        "x": 0,
        "y": 8
      },
      "id": 30,
      "links": [],
      "pageSize": null,
      "scroll": true,
      "showHeader": true,
      "sort": {
        "col": 3,
        "desc": false
      },
      "styles": [
        {
          "alias": "",
          "colorMode": "row",
          "colors": [
            "rgba(50, 172, 45, 0.97)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(245, 54, 54, 0.9)"
          ],
          "decimals": 1,
          "pattern": "When",
          "thresholds": [
            "1800"
          ],
          "type": "number",
          "unit": "s"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "link": true,
          "linkTargetBlank": true,
          "linkTooltip": "${__cell_1}",
          "linkUrl": f"{webhdfs_endpoint}/${{__cell_2}}?op=OPEN",
          "mappingType": 1,
          "pattern": "Canary",
          "thresholds": [],
          "type": "number",
          "unit": "short"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Notebook",
          "thresholds": [],
          "type": "hidden",
          "unit": "short"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Name",
          "thresholds": [],
          "type": "hidden",
          "unit": "short"
        }
      ],
      "targets": [
        {
          "alias": "",
          "format": "table",
          "rawSql": "WITH CTE (session_start, app_name, app_version, name)  \r\nAS  \r\n(  \r\n    SELECT MAX(session_start), app_name, app_version, name as name -- the last step\r\n    FROM metrics\r\n    WHERE parent_name IS NULL -- exclude 'repair' notebook \r\n    AND error_level = 0\r\n    GROUP BY [app_name], [app_version], name\r\n)\r\nSELECT c.app_name as [Canary], MAX(c.name) as [Name], max(r.notebook) as [Notebook], DATEDIFF(s, c.session_start, GETDATE()) as [When] \r\nFROM CTE c\r\nJOIN metrics r on c.session_start = r.session_start and c.app_name = r.app_name and c.app_version = r.app_version and c.name = r.name\r\nGROUP BY c.app_name, c.app_version, c.session_start\r\nORDER BY 4 ASC",
          "refId": "A"
        }
      ],
      "timeFrom": null,
      "timeShift": null,
      "title": "Last success for each App-Deploy",
      "transform": "table",
      "type": "table"
    },
    {
      "columns": [],
      "datasource": "Microsoft SQL Server",
      "fontSize": "100%",
      "gridPos": {
        "h": 6,
        "w": 5,
        "x": 5,
        "y": 8
      },
      "id": 8,
      "links": [],
      "pageSize": null,
      "scroll": true,
      "showHeader": true,
      "sort": {
        "col": 3,
        "desc": false
      },
      "styles": [
        {
          "alias": "",
          "colorMode": "row",
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "When",
          "thresholds": [
            "1800",
            " 3600"
          ],
          "type": "number",
          "unit": "s"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Notebook",
          "thresholds": [],
          "type": "hidden",
          "unit": "short"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "link": true,
          "linkTargetBlank": true,
          "linkTooltip": "${__cell_1}",
          "linkUrl": f"{webhdfs_endpoint}/${{__cell_2}}?op=OPEN",
          "mappingType": 1,
          "pattern": "Canary",
          "thresholds": [],
          "type": "number",
          "unit": "short"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Name",
          "thresholds": [],
          "type": "hidden",
          "unit": "short"
        }
      ],
      "targets": [
        {
          "alias": "",
          "format": "table",
          "rawSql": "WITH CTE (session_start, app_name, app_version, name)  \r\nAS  \r\n(  \r\n    SELECT MAX(session_start), app_name, app_version, name as name -- the last step\r\n    FROM metrics\r\n    WHERE parent_name IS NULL -- exclude 'repair' notebook \r\n    AND error_level <> 0\r\n    GROUP BY [app_name], [app_version], name\r\n)\r\nSELECT c.app_name as [Canary], MAX(c.name) as [Name], max(r.notebook) as [Notebook], DATEDIFF(s, c.session_start, GETDATE()) as [When] \r\nFROM CTE c\r\nJOIN metrics r on c.session_start = r.session_start and c.app_name = r.app_name and c.app_version = r.app_version and c.name = r.name\r\nGROUP BY c.app_name, c.app_version, c.session_start\r\nORDER BY 4 ASC",
          "refId": "A"
        }
      ],
      "timeFrom": null,
      "timeShift": null,
      "title": "Last failure for each App-Deploy",
      "transform": "table",
      "type": "table"
    },
    {
      "columns": [],
      "datasource": "Microsoft SQL Server",
      "fontSize": "100%",
      "gridPos": {
        "h": 6,
        "w": 7,
        "x": 10,
        "y": 8
      },
      "id": 34,
      "links": [],
      "pageSize": null,
      "scroll": true,
      "showHeader": true,
      "sort": {
        "col": 4,
        "desc": false
      },
      "styles": [
        {
          "alias": "",
          "colorMode": "row",
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "decimals": 2,
          "pattern": "Percent",
          "thresholds": [
            "99.9",
            " 100"
          ],
          "type": "number",
          "unit": "percent"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "",
          "thresholds": [],
          "type": "number",
          "unit": "short"
        }
      ],
      "targets": [
        {
          "alias": "",
          "format": "table",
          "rawSql": "SELECT * FROM app_stats(-24)",
          "refId": "A"
        }
      ],
      "timeFrom": null,
      "timeShift": null,
      "title": "Last 24 hours",
      "transform": "table",
      "type": "table"
    },
    {
      "columns": [],
      "datasource": "Microsoft SQL Server",
      "fontSize": "100%",
      "gridPos": {
        "h": 6,
        "w": 7,
        "x": 17,
        "y": 8
      },
      "id": 16,
      "links": [],
      "pageSize": null,
      "scroll": true,
      "showHeader": true,
      "sort": {
        "col": 4,
        "desc": false
      },
      "styles": [
        {
          "alias": "",
          "colorMode": "row",
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "decimals": 2,
          "pattern": "Percent",
          "thresholds": [
            "99.9",
            " 100"
          ],
          "type": "number",
          "unit": "percent"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "",
          "thresholds": [],
          "type": "number",
          "unit": "short"
        }
      ],
      "targets": [
        {
          "alias": "",
          "format": "table",
          "rawSql": "SELECT * FROM app_stats(-1)",
          "refId": "A"
        }
      ],
      "timeFrom": null,
      "timeShift": null,
      "title": "Last hour",
      "transform": "table",
      "type": "table"
    },
    {
      "columns": [],
      "datasource": "Microsoft SQL Server",
      "fontSize": "100%",
      "gridPos": {
        "h": 6,
        "w": 10,
        "x": 0,
        "y": 14
      },
      "id": 10,
      "links": [],
      "pageSize": null,
      "scroll": true,
      "showHeader": true,
      "sort": {
        "col": 0,
        "desc": false
      },
      "styles": [
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "decimals": 2,
          "link": true,
          "linkTargetBlank": true,
          "linkTooltip": "",
          "linkUrl": f"{webhdfs_endpoint}/${{__cell}}?op=OPEN",
          "pattern": "/Output/",
          "thresholds": [],
          "type": "hidden",
          "unit": "short"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Duration",
          "thresholds": [],
          "type": "number",
          "unit": "s"
        },
        {
          "alias": "",
          "colorMode": "value",
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Ago",
          "thresholds": [
            "1800",
            " 3600"
          ],
          "type": "number",
          "unit": "s"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "link": true,
          "linkTooltip": "",
          "linkUrl": f"{webhdfs_endpoint}/${{__cell_4}}?op=OPEN",
          "mappingType": 1,
          "pattern": "Notebook",
          "thresholds": [],
          "type": "string",
          "unit": "short"
        }
      ],
      "targets": [
        {
          "alias": "",
          "format": "table",
          "rawSql": "SELECT datediff(s, start,  GETDATE()) as Ago, datediff(s, start,  [end]) as Duration, app_name as Canary, name as Notebook, notebook as Output FROM metrics \r\nWHERE error_level <> 0 AND start > GETDATE() - 1\r\nORDER BY 1 DESC",
          "refId": "A"
        }
      ],
      "timeFrom": null,
      "timeShift": null,
      "title": "Failures in last 24 hours",
      "transform": "table",
      "type": "table"
    },
    {
      "columns": [],
      "datasource": "Microsoft SQL Server",
      "fontSize": "100%",
      "gridPos": {
        "h": 6,
        "w": 10,
        "x": 10,
        "y": 14
      },
      "id": 38,
      "links": [],
      "pageSize": null,
      "scroll": true,
      "showHeader": true,
      "sort": {
        "col": 0,
        "desc": false
      },
      "styles": [
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "decimals": 2,
          "link": false,
          "linkTargetBlank": false,
          "linkTooltip": "",
          "linkUrl": f"{webhdfs_endpoint}/${{__cell}}?op=OPEN",
          "pattern": "/Output/",
          "thresholds": [],
          "type": "hidden",
          "unit": "short"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Duration",
          "thresholds": [],
          "type": "number",
          "unit": "s"
        },
        {
          "alias": "",
          "colorMode": "value",
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "mappingType": 1,
          "pattern": "Ago",
          "thresholds": [
            "1800",
            " 3600"
          ],
          "type": "number",
          "unit": "s"
        },
        {
          "alias": "",
          "colorMode": null,
          "colors": [
            "rgba(245, 54, 54, 0.9)",
            "rgba(237, 129, 40, 0.89)",
            "rgba(50, 172, 45, 0.97)"
          ],
          "dateFormat": "YYYY-MM-DD HH:mm:ss",
          "decimals": 2,
          "link": true,
          "linkTargetBlank": true,
          "linkTooltip": "",
          "linkUrl": f"{webhdfs_endpoint}/${{__cell_4}}?op=OPEN",
          "mappingType": 1,
          "pattern": "Notebook",
          "thresholds": [],
          "type": "string",
          "unit": "short"
        }
      ],
      "targets": [
        {
          "alias": "",
          "format": "table",
          "rawSql": "SELECT TOP 10 datediff(s, start,  GETDATE()) as Ago, datediff(s, start,  [end]) as Duration, app_name as Canary, name as Notebook, notebook as Output \r\nFROM metrics \r\nWHERE parent_name IS NOT NULL\r\nORDER BY 1 ASC",
          "refId": "A"
        }
      ],
      "timeFrom": null,
      "timeShift": null,
      "title": "Automated Repair Actions (Expert Rules)",
      "transform": "table",
      "type": "table"
    },
    {
      "cacheTimeout": null,
      "colorBackground": false,
      "colorValue": true,
      "colors": [
        "#299c46",
        "rgba(237, 129, 40, 0.89)",
        "#d44a3a"
      ],
      "datasource": "Microsoft SQL Server",
      "format": "s",
      "gauge": {
        "maxValue": 1800,
        "minValue": 0,
        "show": true,
        "thresholdLabels": true,
        "thresholdMarkers": true
      },
      "gridPos": {
        "h": 6,
        "w": 4,
        "x": 20,
        "y": 14
      },
      "id": 6,
      "interval": null,
      "links": [],
      "mappingType": 1,
      "mappingTypes": [
        {
          "name": "value to text",
          "value": 1
        },
        {
          "name": "range to text",
          "value": 2
        }
      ],
      "maxDataPoints": 100,
      "nullPointMode": "connected",
      "nullText": null,
      "pluginVersion": "6.1.6",
      "postfix": "",
      "postfixFontSize": "50%",
      "prefix": "",
      "prefixFontSize": "50%",
      "rangeMaps": [
        {
          "from": "null",
          "text": "N/A",
          "to": "null"
        }
      ],
      "sparkline": {
        "fillColor": "rgba(31, 118, 189, 0.18)",
        "full": true,
        "lineColor": "rgb(31, 120, 193)",
        "show": true
      },
      "tableColumn": "Last Reading (seconds])",
      "targets": [
        {
          "alias": "",
          "format": "table",
          "rawSql": "SELECT TOP 1 DATEDIFF(s, max(start), GETDATE()) as 'Last Reading (seconds])', app_name\r\nFROM metrics\r\nWHERE parent_name IS NULL\r\nGROUP BY [app_name], [name]\r\nORDER BY 1 DESC",
          "refId": "A"
        }
      ],
      "thresholds": "1200, 1500, 1800",
      "timeFrom": null,
      "timeShift": null,
      "title": "Freshness",
      "transparent": true,
      "type": "singlestat",
      "valueFontSize": "50%",
      "valueMaps": [
        {
          "op": "=",
          "text": "N/A",
          "value": "null"
        }
      ],
      "valueName": "avg"
    }
  ],
  "refresh": "10s",
  "schemaVersion": 18,
  "style": "dark",
  "tags": [],
  "templating": {
    "list": []
  },
  "time": {
    "from": "now-3h",
    "to": "now"
  },
  "timepicker": {
    "refresh_intervals": [
      "5s",
      "10s",
      "30s",
      "1m",
      "5m",
      "15m",
      "30m",
      "1h",
      "2h",
      "1d"
    ],
    "time_options": [
      "5m",
      "15m",
      "1h",
      "6h",
      "12h",
      "24h",
      "2d",
      "7d",
      "30d"
    ]
  },
  "timezone": "",
  "title": f"Big Data Cluster Notebook App-Deploy Monitors ({namespace})",
  "uid": "Le-84yaWz",
  "version": null
}

payload = {"dashboard": dashboard, "overwrite": True}

### Get name of the ‘Running’ `controller` `pod`

In [ ]:
# Place the name  of the 'Running' controller pod in variable `controller`

controller = run(f'kubectl get pod --selector=app=controller -n {namespace} -o jsonpath={{.items[0].metadata.name}} --field-selector=status.phase=Running', return_output=True)

print(f"Controller pod name: {controller}")

### Is notebook being run inside a Kubernetes cluster

When this is notebook is running inside a Kubernetes cluster, such as
when running inside an App-Deploy pod, there is no KUBECONFIG present,
therefore azdata login needs to use the -e (endpoint) approach to login.

In [ ]:
import os

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    inside_kubernetes_cluster = True
else:
    inside_kubernetes_cluster = False

### Get `metricsui` endpoint

In [ ]:
import json

if inside_kubernetes_cluster:
    metricsui = "metricsui-svc,3000"
else:
    endpoint = run('azdata bdc endpoint list --endpoint="metricsui"', return_output=True)
    endpoint = json.loads(endpoint)
    metricsui  = endpoint['endpoint']

print (f"The metricsui endpoint: {metricsui }")

### Create a temporary directory to stage files

In [ ]:
# Create a temporary directory to hold configuration files

import tempfile

temp_dir = tempfile.mkdtemp()

print(f"Temporary directory created: {temp_dir}")

### Copy Self Signed Root CA locally

NOTE: Grafana seems to only work with the `urllib3` library, which seems
to require the Root CA certificate bundle to be loaded explciltly. Even
if a self signed Root CA certificate is loaded in the local certificate
store (and works fine in Chrome and Edge etc.). Using the `requests`
library causes Grafana to return a 400. Using the urllib3 library
without creating a specific SSLContextAdapter adapter, causes SSL
verification to fail.

In [ ]:
cwd = os.getcwd()
os.chdir(temp_dir) # Workaround kubectl bug on Windows, can't put c:\ on kubectl cp cmd line 

run(f'kubectl cp {controller}:{test_cert_store_root}/cacert.pem cacert.crt -c controller -n {namespace}')

os.chdir(cwd)

print(f'Root CA certificate copied locally to: {temp_dir}')

### Setup HTTP SSL Session

In [ ]:
import os, json, urllib
from urllib.request import Request, urlopen, HTTPPasswordMgrWithDefaultRealm, HTTPBasicAuthHandler, build_opener, install_opener

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.ssl_ import create_urllib3_context

url = urllib.parse.urlparse(f"{metricsui}/api")

headers = {}
headers['Content-Type'] = 'application/json'
headers['Accept'] = 'application/json'

# Load Root CA certificates
#
#     https://stackoverflow.com/questions/42981429/ssl-failure-on-windows-using-python-requests/50215614
#
class SSLContextAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = create_urllib3_context()
        kwargs['ssl_context'] = context

        context.load_default_certs() # this loads the OS defaults on Windows

        if os.path.isfile(os.path.join(temp_dir, "cacert.crt")):
            context.load_verify_locations(os.path.join(temp_dir, "cacert.crt")) # load self-signed Root CA if it exists

        return super(SSLContextAdapter, self).init_poolmanager(*args, **kwargs)

session = requests.Session()
adapter = SSLContextAdapter()
session.mount(f'{url.scheme}://{url.hostname}', adapter)

### Create Datasource in Grafana

The Grafana Datasource API documentation:

-   https://grafana.com/docs/http\_api/data\_source/

The SQL Server Datasource documentation:

-   https://grafana.com/docs/features/datasources/mssql/

In [ ]:
response = session.post(url=f'{url.geturl()}/datasources', auth=(os.environ["AZDATA_USERNAME"], os.environ["AZDATA_PASSWORD"]), headers=headers, data=json.dumps(datasource))

print(response)
print(response.text)

### Create Dashboard in Grafana

The Grafana Dashboard API documentation:

-   https://grafana.com/docs/http\_api/dashboard/

In [ ]:
response = session.post(url=f'{url.geturl()}/dashboards/db', auth=(os.environ["AZDATA_USERNAME"], os.environ["AZDATA_PASSWORD"]), headers=headers, data=json.dumps(payload))

print(response)
print(response.text)

if response.status_code == 200:
    j = json.loads(response.text)
    url = f'{url.scheme}://{url.hostname}:{url.port}{j["url"]}'

    print("")
    print(f'Dashboard URL: {url}')
else:
    if response.text.find("Dashboard not found") > -1:
        raise SystemExit('NOTE: "Dashboard not found" can be caused by not setting the "id" to "null" in the dashboard definition above.')
    else:
        raise SystemExit('There was an error POSTing the dashboard to: ' + f'{url.geturl()}/dashboards/db')

### Load new dashboard in Web Browser

In [ ]:
import webbrowser

if response.status_code:
    webbrowser.open(url)

### Clean up temporary directory for staging configuration files

In [ ]:
# Delete the temporary directory used to hold configuration files

import shutil

shutil.rmtree(temp_dir)

print(f'Temporary directory deleted: {temp_dir}')

### Create Notebook App Stats table valued function

In [ ]:
sql = f"""
IF OBJECT_ID('dbo.app_stats') IS NULL
    EXEC dbo.sp_executesql @statement = N'CREATE FUNCTION [dbo].[app_stats] (@hours INT) RETURNS TABLE AS RETURN SELECT 1 as dummy_function'
"""

run(f'azdata sql query --database runner -q "{sql}"')

sql = f"""
ALTER FUNCTION app_stats (@hours INT)
RETURNS TABLE
AS
RETURN
    WITH last_hours([app_name], [passed], [failed])
    AS
    (
        SELECT  [app_name],
            (SELECT count(*) FROM [metrics] r2 WHERE r1.app_name = r2.app_name AND r2.error_level = 0 AND start > DATEADD(hour, @hours, GETDATE())) AS [passed],
            (SELECT count(*) FROM [metrics] r2 WHERE r1.app_name = r2.app_name AND r2.error_level <> 0 AND start > DATEADD(hour, @hours, GETDATE())) AS [failed]
        FROM [metrics] r1
        WHERE start > DATEADD(hour, @hours, GETDATE())
        GROUP BY [app_name]
    )
    SELECT [app_name] as Canary, [Passed], [Failed], [passed] + [failed] as Total, CONVERT(DECIMAL(5, 2), ([passed] * 1.0) / ([passed] + [failed]) * 100.0) as [Percent]
    FROM last_hours
"""

run(f'azdata sql query --database runner -q "{sql}"')

In [ ]:
print('Notebook execution complete.')

Related
-------

-   [OPR100 - Deploy and Schedule
    notebook(s)](../notebook-o16n/opr100-deploy-and-schedule-notebook.ipynb)